In [1]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
import holoviews as hv
import base as b
import hvplot.pandas  # noqa
from functools import cache
from logger_settings import logger
from constants import *
from bokeh.plotting import figure, show, output_notebook
output_notebook()


test_date = dt.datetime.strptime("2024-01-01", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE


TEST_START = dt.datetime.strptime("2024-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2024-02-15", "%Y-%m-%d")

train_dates, test_dates = ut.get_date_range(start_date=TEST_START, end_date=TEST_END, symbol=SYMBOL, interval=INTERVAL, exchange=EXCHANGE)
train_dates["expiry"] = pd.NA
train_dates["expiry"] = train_dates.apply(lambda row: ut.find_nclosest_expiry(SYMBOL, row.name, 1), axis=1)

train_dates['call_strike'] = train_dates.apply(lambda r: ut.get_strike_price_by_price(symbol=SYMBOL, expiry=r.expiry, timestamp=r.name.replace(hour=9, minute=15), option_type=OPTION_TYPE_CALL, price=30, exchange=EXCHANGE), axis=1)
train_dates['put_strike'] = train_dates.apply(lambda r: ut.get_strike_price_by_price(symbol=SYMBOL, expiry=r.expiry, timestamp=r.name.replace(hour=9, minute=15), option_type=OPTION_TYPE_PUT, price=30, exchange=EXCHANGE), axis=1)

# Get nifty candles for minute
# For first minute, get premium strike price for call and puts which is price at ~30 
# Draw them here 
# Try to match the pattern 

/home/nitin/nse-reports-trade/utils.py:39: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  scrip_df = pd.read_csv(file_path)


Loading BokehJS ...

In [57]:
settings = {
    "VOLUME_QUANTILE_THRESHOLD": 10 / 100, # Choose least this % volume from the given volumes
    "CANDLE_LENTH": 20,
    "MIN_GREEN_CANDLE_LENGTH": 3, # Minimum candles to be considered as pattern
    "RED_CANDLE_RATIO": 0.65,
    "GREEN_THRESHOLD": 0.05, # For candle to be considered green, percentage of average
    "RED_CANDLE_DIFF": 0.20, # Red candle size compared to green up move
    "MIN_QV_THRESHOLD": 3.8, # Minimum volume ratio to qualify for green candle
    # Order Settings
    "quantity": 25,
    "TARGET_PC": 0.05,
    "STOPLOSS_PC": 0.03,
}

from typing import Dict


class PeakStrategy(b.Strategy):
    def __init__(self, instrument: "Instrument", settings: Dict):
        super().__init__(instrument, settings)
        self.call_ticks: pd.DataFrame | None = None
        self.put_ticks: pd.DataFrame | None = None
        self.om = b.OrderManager()

    def calculate_data(self):
        self.put_ticks['qv'] = self.put_ticks['volume'].rolling(window=self.CANDLE_LENTH).quantile(self.VOLUME_QUANTILE_THRESHOLD)
        self.put_ticks['qv_ratio'] = self.put_ticks.volume / self.put_ticks.qv
        self.put_ticks['qv_qualify'] = self.put_ticks.qv_ratio > self.MIN_QV_THRESHOLD
        self.put_ticks['prev_ratio'] = self.put_ticks.volume / self.put_ticks.volume.shift(1)
        self.put_ticks['prev1_ratio'] = self.put_ticks.volume / self.put_ticks.volume.shift(2)
        self.put_ticks['prev_qv_qualify'] = self.put_ticks.qv_qualify.shift(1)
        self.put_ticks['prev1_qv_qualify'] = self.put_ticks.qv_qualify.shift(2)
        self.put_ticks['red_qualify'] = (self.put_ticks.prev_qv_qualify | self.put_ticks.prev1_qv_qualify) & ((self.put_ticks.prev_ratio < self.RED_CANDLE_RATIO) | (self.put_ticks.prev1_ratio < self.RED_CANDLE_RATIO))
        self.put_ticks['wprice'] = (self.put_ticks.high + self.put_ticks.low + 2 * self.put_ticks.close) / 4
        self.call_ticks['wprice'] = (self.call_ticks.high + self.call_ticks.low + 2 * self.call_ticks.close) / 4
        self.put_ticks['cdiff'] = self.put_ticks.wprice.diff()
        self.put_ticks['is_green'] = self.put_ticks.cdiff > 0
        self.put_ticks['is_small'] = self.put_ticks.cdiff < self.put_ticks.cdiff.rolling(window=self.CANDLE_LENTH).mean() * self.GREEN_THRESHOLD

    def is_volume_match(self) -> bool:
        '''
        1. Must have high volume on the last green candle
        2. Must have low volume on the last red candle
        '''
        return self.put_ticks.iloc[-1].red_qualify

    def is_price_match(self):
        '''
        1. Price must be increasing until the last candle
        2. There must not be big gaps in the candles, that could mean that it's going to go further up
        '''
        last_tick = self.put_ticks.iloc[-1]
        last_n_ticks = self.put_ticks[-self.MIN_GREEN_CANDLE_LENGTH-1:-1]
        green_length = last_n_ticks.loc[last_n_ticks.is_green].shape[0]
        if green_length < self.MIN_GREEN_CANDLE_LENGTH:
            last_n_ticks = self.put_ticks[-self.MIN_GREEN_CANDLE_LENGTH-2:-1]
        if last_tick.is_green:
            return False
        if green_length < self.MIN_GREEN_CANDLE_LENGTH - 1:
            return False
        elif green_length == self.MIN_GREEN_CANDLE_LENGTH:
            return True
        price_min, price_max = last_n_ticks.close.min(), last_n_ticks.close.max()
        red_ratio = (price_max - last_tick.close) / (price_max - price_min)
        if red_ratio < self.RED_CANDLE_DIFF:
            return True
        return False

    def entry_conditions(self) -> bool:
        last_tick = self.put_ticks.iloc[-1]
        last_n_ticks = self.put_ticks[-self.MIN_GREEN_CANDLE_LENGTH-1:-1]
        is_price_match = self.is_price_match()
        is_volume_match = self.is_volume_match()
        if not is_volume_match:
            return False
        if not is_price_match:
            return False
        print(f"matched at: {self.put_ticks.iloc[-1].timestamp}")
        return True

    def exit_conditions(self) -> bool:
        last_tick = self.call_ticks.iloc[-1]
        wprice = last_tick.wprice
        openp = last_tick.open
        closep = last_tick.close
        for order in self.om.orders:
            if order.created_at == last_tick.timestamp:
                continue
            if ((order.limit_price - openp) / order.limit_price) >= self.STOPLOSS_PC:
                return True, openp
            if ((wprice - order.limit_price) / order.limit_price) >= self.TARGET_PC:
                return True, wprice
            if ((openp - order.limit_price) / order.limit_price) >= self.TARGET_PC:
                return True, openp
            if ((order.limit_price - closep) / order.limit_price) >= self.STOPLOSS_PC:
                return True, closep
        return False, None

    def _process_tick(self, call_tick: Dict, put_tick: Dict) -> None:
        call_tick_df = pd.DataFrame(
            call_tick, index=[len(self.call_ticks) if self.call_ticks is not None else 0]
        )
        self.call_ticks = pd.concat([self.call_ticks, call_tick_df], ignore_index=True)

        put_tick_df = pd.DataFrame(
            put_tick, index=[len(self.put_ticks) if self.put_ticks is not None else 0]
        )
        self.put_ticks = pd.concat([self.put_ticks, put_tick_df], ignore_index=True)

    def next(self, call_tick: Dict, put_tick: Dict):
        self._process_tick(call_tick, put_tick)
        self.calculate_data()
        if not self.om.has_intrade_orders() and self.entry_conditions():
            order = b.Order(type=b.Order.TYPE_BUY, limit_price=self.call_ticks.iloc[-1].close, created_at=self.call_ticks.iloc[-1].timestamp, quantity=self.quantity, exchange_order_id=None)
            self.om.place_order(order)
        is_exit, eprice = self.exit_conditions()
        if self.om.has_intrade_orders() and is_exit:
            self.om.square_off_all_orders(index=self.call_ticks.iloc[-1].timestamp, last_price=eprice)


instrument = b.Instrument(name="NIFTY 22650 CALL 7 Mar 2024")

ps = PeakStrategy(instrument=instrument, settings=settings)

for index, row in train_dates.iterrows():
    call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=index.date(), strike_price=row.call_strike, option_type=OPTION_TYPE_CALL)
    put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=row.expiry, cur_dt=index.date(), strike_price=row.put_strike, option_type=OPTION_TYPE_PUT)
    for i in range(put_df.shape[0]):
        cdict = call_df.iloc[i].to_dict()
        cdict['timestamp'] = call_df.iloc[i].name
        pdict = put_df.iloc[i].to_dict()
        pdict['timestamp'] = put_df.iloc[i].name
        cdict, pdict = pdict, cdict
        ps.next(cdict, pdict)
    break

matched at: 2024-01-01 12:15:00
matched at: 2024-01-01 13:04:00
matched at: 2024-01-01 13:59:00
matched at: 2024-01-01 14:06:00
matched at: 2024-01-01 14:51:00


In [58]:
pcs = [order.pnl_pc for order in ps.om.closed_orders]
print(f"PnL Per Order: {sum(pcs) / len(pcs)}")
pcs

PnL Per Order: 3.000117309406347


[5.465116279069754,
 -3.4482758620689618,
 -6.250000000000008,
 5.263157894736834,
 13.970588235294118]

In [56]:
ps.om.closed_orders[-2]

buy, at:2024-01-01 14:10:00, b:34.45, sqat:2024-01-01 14:11:00, s:36.275, pnl:5.297532656023209

In [19]:
ut.create_candlestick_plot(call_df)
ut.create_candlestick_plot(put_df)

GridPlot(id='p3108', ...)

In [40]:
ps.put_ticks.hvplot(x='timestamp', y='cdiff')

:Curve   [timestamp]   (cdiff)

In [30]:
pd.set_option('display.max_rows', 200)
ps.put_ticks[["timestamp", "qv", "qv_ratio", "qv_qualify", "prev_ratio", "prev1_ratio", "prev_qv_qualify", "prev1_qv_qualify"]].head(200)

,timestamp,qv,qv_ratio,qv_qualify,prev_ratio,prev1_ratio,prev_qv_qualify,prev1_qv_qualify
0,2024-01-01 09:15:00,NaN,NaN,False,NaN,NaN,NaN,NaN
1,2024-01-01 09:16:00,NaN,NaN,False,0.653924,NaN,False,NaN
2,2024-01-01 09:17:00,NaN,NaN,False,2.042721,1.335784,False,False
3,2024-01-01 09:18:00,NaN,NaN,False,1.654100,3.378865,False,False
4,2024-01-01 09:19:00,NaN,NaN,False,0.725503,1.200055,False,False
5,2024-01-01 09:20:00,NaN,NaN,False,0.823550,0.597488,False,False
6,2024-01-01 09:21:00,NaN,NaN,False,0.626201,0.515707,False,False
7,2024-01-01 09:22:00,NaN,NaN,False,0.480658,0.300988,False,False
8,2024-01-01 09:23:00,NaN,NaN,False,1.069380,0.514006,False,False
9,2024-01-01 09:24:00,NaN,NaN,False,0.762111,0.814986,False,False
